In [2]:
import os
import matplotlib.pyplot as plt
import argparse
import numpy as np

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D, Flatten, Reshape, Add, Concatenate, MaxPool2D
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from keras import backend as K

from model.unet import unet, unet_small, dice_coef, normalized_iou_coef, hard_normalized_hard_iou_coef
from model.fcn import fcn_8s
from model.pspnet import pspnet50
from dataset_parser import generator_miscl, generator

from scripts import *

%load_ext autoreload
%autoreload 2

print('GPUs available:', K.tensorflow_backend._get_available_gpus())

CLEAN_N = 500
TRAIN_N = 2975
VAL_N = 500

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
GPUs available: ['/job:localhost/replica:0/task:0/device:GPU:0']


# Clean Model

In [2]:
# TRAIN_BATCH = int(4)
# VAL_BATCH = int(4)
# lr_init = 1e-4
# lr_decay = 5e-4
# vgg_path = None
# TRAIN_N = 2975
# VAL_N = 500

# model_name = "modular_unet_clean_benchmark_other_metrics.h5"

# labels = ['background', 'car', 'road']  # Use only 3 classes.
# ignore_pedestrians=not('person' in labels)

# clean_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
#                     lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)

In [3]:
# checkpoint = ModelCheckpoint(filepath=model_name,
#                              monitor='val_dice_coef',
#                              mode='max',
#                              save_best_only=True,
#                              save_weights_only=True)

In [4]:
# history = clean_model.fit_generator(generator.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians),
#                                     steps_per_epoch= TRAIN_N // TRAIN_BATCH,
#                                     validation_data=generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
#                                     validation_steps= VAL_N // VAL_BATCH,
#                                     callbacks=[checkpoint],
#                                     epochs=5, verbose=1)

In [5]:
# clean_model.load_weights(model_name)

# Just Gold Data

In [9]:
TRAIN_BATCH = int(4)
VAL_BATCH = int(4)
lr_init = 1e-4
lr_decay = 5e-4
vgg_path = None
TRAIN_N = 2975
VAL_N = 500

model_name = "modular_unet_gold_benchmark_other_metrics.h5"

labels = ['background', 'car', 'road']  # Use only 3 classes.
ignore_pedestrians=not('person' in labels)

gold_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
                    lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)

In [10]:
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_dice_coef',
                             mode='max',
                             save_best_only=True,
                             save_weights_only=True)

In [11]:
fine_generator = generator.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians)

history = gold_model.fit_generator(fine_generator,
                                   steps_per_epoch= CLEAN_N // TRAIN_BATCH,
                                   validation_data=generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
                                   validation_steps= VAL_N // VAL_BATCH,
                                   callbacks=[checkpoint],
                                   epochs=5, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
2975
2975
500
500


125/125 [==============================] - 63s 502ms/step - loss: 0.4749 - dice_coef: 0.7137 - normalized_iou_coef: 0.4246 - hard_normalized_hard_iou_coef: 0.4976 - val_loss: 0.5496 - val_dice_coef: 0.7353 - val_normalized_iou_coef: 0.4424 - val_hard_normalized_hard_iou_coef: 0.4723
Epoch 2/5
125/125 [==============================] - 52s 417ms/step - loss: 0.3077 - dice_coef: 0.8140 - normalized_iou_coef: 0.5392 - hard_normalized_hard_iou_coef: 0.6393 - val_loss: 0.3637 - val_dice_coef: 0.8228 - val_normalized_iou_coef: 0.5757 - val_hard_normalized_hard_iou_coef: 0.6411
Epoch 3/5
125/125 [==============================] - 53s 427ms/step - loss: 0.2848 - dice_coef: 0.8360 - normalized_iou_coef: 0.6073 - hard_normalized_hard_iou_coef: 0.7033 - val_loss: 0.3476 - val_dice_coef: 0.8418 - val_normalized_iou_coef: 0.6050 - val_hard_normalized_hard_iou_coef: 0.6639
Ep

# Pseudolabeling

In [12]:
TRAIN_BATCH = int(4)
VAL_BATCH = int(4)
lr_init = 1e-4
lr_decay = 5e-4
vgg_path = None

model_name = "modular_unet_pseu_benchmark_other_metrics_miscl50.h5"

labels = ['background', 'car', 'road']  # Use only 3 classes.
ignore_pedestrians=not('person' in labels)

pseu_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
                    lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)

In [13]:
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_dice_coef',
                             mode='max',
                             save_best_only=True,
                             save_weights_only=True)

In [14]:
n_epochs = 3

for e in range(n_epochs):
    miscl_generator = generator_miscl.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians, misclassified_frac=0.5)
    
    for i in range(TRAIN_N // TRAIN_BATCH):
        x, _ = next(miscl_generator)
        y_pred = gold_model.predict(x)
        metrics = pseu_model.train_on_batch(x, y_pred)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = pseu_model.evaluate_generator(generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
                              steps=VAL_N // VAL_BATCH)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))

2975
2975
0 [1.283016, 0.3212763, 0.1609953, 0.1447708450200595]
1 [1.13935, 0.36071455, 0.17979054, 0.21901636132960137]
2 [1.0512354, 0.39209756, 0.19764239, 0.2691530205375411]
3 [1.0010638, 0.40923157, 0.20582424, 0.28688198862348435]
4 [0.9164231, 0.44916937, 0.23502629, 0.35271572835176423]
5 [0.9124046, 0.4592053, 0.23254791, 0.31985320826802355]
6 [0.79359776, 0.50647867, 0.2660824, 0.40409495183922456]
7 [0.7456655, 0.5288509, 0.28713542, 0.44052654080266]
8 [0.8347912, 0.5017437, 0.2675411, 0.3778168830644377]
9 [0.7458923, 0.53739446, 0.2827358, 0.4172306160753183]
10 [0.7936506, 0.5163996, 0.25710577, 0.33811419826816524]
11 [0.720214, 0.55236787, 0.28719842, 0.40769486002937444]
12 [0.67457473, 0.5751532, 0.30029055, 0.4283477911085812]
13 [0.74307793, 0.5496057, 0.2729519, 0.36570971190700546]
14 [0.73674977, 0.5497603, 0.26834208, 0.37197122312412056]
15 [0.6616156, 0.5713579, 0.28201652, 0.40585141787156565]
16 [0.654894, 0.58076894, 0.2904642, 0.4109459871737555]
17 [0

# Coarse Data

In [15]:
TRAIN_BATCH = int(4)
VAL_BATCH = int(4)
lr_init = 1e-4
lr_decay = 5e-4
vgg_path = None
TRAIN_N = 2975
VAL_N = 500

model_name = "modular_unet_misc50_benchmark_other_metrics.h5"

labels = ['background', 'car', 'road']  # Use only 3 classes.
ignore_pedestrians=not('person' in labels)

coarse_model = unet(input_shape=(256, 512, 3), num_classes=len(labels),
                    lr_init=lr_init, lr_decay=lr_decay, vgg_weight_path=vgg_path)

In [16]:
checkpoint = ModelCheckpoint(filepath=model_name,
                             monitor='val_dice_coef',
                             mode='max',                             
                             save_best_only=True,
                             save_weights_only=True)

In [17]:
miscl_generator = generator_miscl.data_generator('dataset_parser/data.h5', TRAIN_BATCH, 'train', ignore_pedestrians=ignore_pedestrians, misclassified_frac=0.5)

In [18]:
history = coarse_model.fit_generator(miscl_generator,
                                     steps_per_epoch= TRAIN_N // TRAIN_BATCH,
                                     validation_data=generator.data_generator('dataset_parser/data.h5', VAL_BATCH, 'val', ignore_pedestrians=ignore_pedestrians),
                                     validation_steps=VAL_N // VAL_BATCH,
                                     callbacks=[checkpoint],
                                     epochs=5, verbose=1)

Epoch 1/5
500
500
2975
2975
743/743 [==============================] - 255s 344ms/step - loss: 0.3086 - dice_coef: 0.8211 - normalized_iou_coef: 0.5082 - hard_normalized_hard_iou_coef: 0.5536 - val_loss: 0.5160 - val_dice_coef: 0.7589 - val_normalized_iou_coef: 0.4749 - val_hard_normalized_hard_iou_coef: 0.4758
Epoch 2/5
743/743 [==============================] - 249s 336ms/step - loss: 0.2180 - dice_coef: 0.8768 - normalized_iou_coef: 0.5763 - hard_normalized_hard_iou_coef: 0.6101 - val_loss: 0.2957 - val_dice_coef: 0.8573 - val_normalized_iou_coef: 0.5730 - val_hard_normalized_hard_iou_coef: 0.5621
Epoch 3/5
743/743 [==============================] - 250s 337ms/step - loss: 0.1952 - dice_coef: 0.8901 - normalized_iou_coef: 0.5970 - hard_normalized_hard_iou_coef: 0.6282 - val_loss: 0.2696 - val_dice_coef: 0.8683 - val_normalized_iou_coef: 0.6123 - val_hard_normalized_hard_iou_coef: 0.6875
Epoch 4/5
743/743 [==============================] - 252s 339ms/step - loss: 0.1785 - dice_coef: 

# Apply Error Correction Networks (Full)

In [20]:
from keras.layers import Input, Embedding, LSTM, Dense, Conv2D, Flatten, Reshape, Add, Concatenate, MaxPool2D, Lambda
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from keras import backend as K

input_window_size = 64
output_window_size = 1

# ec_model = get_ec_model(input_window_size, output_window_size, dice_coef=dice_coef)

lr_init = 1e-3
lr_decay = 5e-4

inp_y = Input(shape=(input_window_size, input_window_size, 3))
inp_x = Input(shape=(input_window_size, input_window_size, 3))

h = Concatenate()([inp_x, inp_y])
h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
h = MaxPool2D((2, 2), padding='same')(h)
h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
h = MaxPool2D((2, 2), padding='same')(h)
h = Conv2D(16, (4, 4), padding='same', activation='relu')(h)
h = Flatten()(h)
h = Dense(40, activation='relu')(h)
h = Dense(3, activation='softmax')(h)
h = Reshape((1, 1, 3))(h)
ec_model = Model(inputs=[inp_x, inp_y], outputs=h)

ec_model.summary()
metrics=[dice_coef, normalized_iou_coef, hard_normalized_hard_iou_coef]
ec_model.compile(optimizer=Adam(lr=lr_init, decay=lr_decay),
                 loss='categorical_crossentropy', metrics=metrics)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
concatenate_26 (Concatenate)    (None, 64, 64, 6)    0           input_10[0][0]                   
                                                                 input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 64, 64, 8)    776         concatenate_26[0][0]             
__________

In [21]:
n_epochs = 3
batch_size = 8
skip = 12
N_CLEAN = 500
best_val = 0

val_generator = generator.data_generator('dataset_parser/data.h5', 32, 'val', ignore_pedestrians=True)

x_val, y_val = next(val_generator)
y_val_coarse = coarse_model.predict(x_val)
x_val_batch, y_val_coarse_batch, y_val_diff_batch = get_ec_batch(
    x_val, y_val, y_val_coarse, input_window_size=input_window_size,
    output_window_size=output_window_size, skip=skip, diff=False)

for e in range(n_epochs):
    fine_generator = generator.data_generator('dataset_parser/data.h5', batch_size, 'train', 
                                              ignore_pedestrians=ignore_pedestrians)
    for i in range(N_CLEAN // batch_size):
        x, y_fine = next(fine_generator)
        y_coarse = coarse_model.predict(x)
        x_batch, y_coarse_batch, y_diff_batch = get_ec_batch(x, y_fine, y_coarse, input_window_size=input_window_size,
                                                             output_window_size=output_window_size, skip=skip, diff=False,
                                                             balanced=True)
        
#         metrics = ec_model.train_on_batch(y_coarse_batch, y_diff_batch)
        metrics = ec_model.train_on_batch([x_batch, y_coarse_batch], y_diff_batch)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = ec_model.evaluate([x_val_batch, y_val_coarse_batch], y_val_diff_batch)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))
    if metrics[-1] > best_val:
        best_val = metrics[-1]
        ec_model.save('best-ec-model-5.h5')
        print('Model saved')

500
500
2975
2975
0 [1.0899742, 0.33670714, 0.19550888, 0.15528557778108432]
1 [0.96955115, 0.38946182, 0.21867125, 0.2105567728524074]
2 [0.9329871, 0.43717414, 0.24484996, 0.18591714387147246]
3 [0.94355804, 0.4629965, 0.2337117, 0.17301442453932728]
4 [0.75331414, 0.52321035, 0.30355176, 0.41914529914529913]
5 [0.6578739, 0.562662, 0.32995638, 0.5014883853831593]
6 [0.70798296, 0.5353167, 0.31905192, 0.45151804873818646]
7 [0.61641383, 0.5943158, 0.37118596, 0.4950837129516691]
8 [0.67526734, 0.58858234, 0.4035513, 0.4084517897298624]
9 [0.62616163, 0.64681846, 0.44685555, 0.5104074220259116]
10 [0.6447356, 0.6520828, 0.44379342, 0.6439514910194771]
11 [0.6786576, 0.65402967, 0.42175254, 0.5982096102076189]
12 [0.5881129, 0.6693047, 0.46196562, 0.601380405738349]
13 [0.56052035, 0.67614156, 0.48068818, 0.6704865439127282]
14 [0.4925555, 0.71285385, 0.4876238, 0.6539942170253015]
15 [0.51331156, 0.7187136, 0.44814616, 0.5380175545477108]
16 [0.50572646, 0.7342521, 0.49764824, 0.58342

In [22]:
y_p = ec_model.predict([x_val_batch, y_val_coarse_batch])

In [24]:
np_hard_normalized_hard_iou_coef(y_val_diff_batch, y_p)

0.7915144647654407

# Apply Error Correction Networks (Y only)

In [27]:
def get_ec_model():
    input_window_size = 64
    output_window_size = 1

    # ec_model = get_ec_model(input_window_size, output_window_size, dice_coef=dice_coef)

    lr_init = 1e-3
    lr_decay = 5e-4

    inp_y = Input(shape=(input_window_size, input_window_size, 3))
    inp_x = Input(shape=(input_window_size, input_window_size, 3))

    h = Concatenate()([inp_x, inp_y])
    h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
    h = MaxPool2D((2, 2), padding='same')(h)
    h = Conv2D(8, (4, 4), padding='same', activation='relu')(h)
    h = MaxPool2D((2, 2), padding='same')(h)
    h = Conv2D(16, (4, 4), padding='same', activation='relu')(h)
    h = Flatten()(h)
    h = Dense(40, activation='relu')(h)
    h = Dense(3, activation='softmax')(h)
    h = Reshape((1, 1, 3))(h)
    ec_model = Model(inputs=[inp_x, inp_y], outputs=h)

    ec_model.summary()
    metrics=[dice_coef, normalized_iou_coef, hard_normalized_hard_iou_coef]
    ec_model.compile(optimizer=Adam(lr=lr_init, decay=lr_decay),
                     loss='categorical_crossentropy', metrics=metrics) 
    return ec_model

In [28]:
ec_model = get_ec_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
concatenate_27 (Concatenate)    (None, 64, 64, 6)    0           input_12[0][0]                   
                                                                 input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 64, 64, 8)    776         concatenate_27[0][0]             
__________

In [29]:
n_epochs = 3
batch_size = 8
skip = 12
N_CLEAN = 500
best_val = 0

val_generator = generator.data_generator('dataset_parser/data.h5', 32, 'val', ignore_pedestrians=True)

x_val, y_val = next(val_generator)
y_val_coarse = coarse_model.predict(x_val)
x_val_batch, y_val_coarse_batch, y_val_diff_batch = get_ec_batch(
    x_val, y_val, y_val_coarse, input_window_size=input_window_size,
    output_window_size=output_window_size, skip=skip, diff=False)

for e in range(n_epochs):
    fine_generator = generator.data_generator('dataset_parser/data.h5', batch_size, 'train', 
                                              ignore_pedestrians=ignore_pedestrians)
    for i in range(N_CLEAN // batch_size):
        x, y_fine = next(fine_generator)
        y_coarse = coarse_model.predict(x)
        x_batch, y_coarse_batch, y_diff_batch = get_ec_batch(x, y_fine, y_coarse, input_window_size=input_window_size,
                                                             output_window_size=output_window_size, skip=skip, diff=False,
                                                             balanced=True)
        x_batch = np.random.random(x_batch.shape)
#         metrics = ec_model.train_on_batch(y_coarse_batch, y_diff_batch)
        metrics = ec_model.train_on_batch([x_batch, y_coarse_batch], y_diff_batch)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = ec_model.evaluate([x_val_batch, y_val_coarse_batch], y_val_diff_batch)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))
    if metrics[-1] > best_val:
        best_val = metrics[-1]
        ec_model.save('best-ec-model-5.h5')
        print('Model saved')

500
500
2975
2975
0 [1.0638752, 0.34604466, 0.20250788, 0.36575460259916837]
1 [0.8706744, 0.4466419, 0.25641018, 0.4179366912012315]
2 [0.7745113, 0.53865945, 0.32892856, 0.4195259419046746]
3 [0.788637, 0.576987, 0.31355312, 0.3364144833676773]
4 [0.6247183, 0.6344061, 0.4006551, 0.4335699564918709]
5 [0.5623673, 0.6722417, 0.41574416, 0.44622203393556875]
6 [0.6794204, 0.6184438, 0.38782904, 0.42407717733521677]
7 [0.54980737, 0.68270266, 0.44190437, 0.4591397280033489]
8 [0.70625246, 0.62728715, 0.4320876, 0.412389646301522]
9 [0.68001705, 0.6358811, 0.43934307, 0.41551562569732675]
10 [0.6819064, 0.6232799, 0.41516298, 0.5810959099357846]
11 [0.7347034, 0.61780894, 0.38249797, 0.4718390604074836]
12 [0.5850553, 0.6755503, 0.44199586, 0.6109368966526301]
13 [0.559733, 0.6761083, 0.45471483, 0.598147641054145]
14 [0.5449826, 0.69600797, 0.44542, 0.4933532794855573]
15 [0.51034325, 0.7213093, 0.43840027, 0.4536522982020612]
16 [0.57530403, 0.70798486, 0.44182953, 0.46244438005612665]

In [30]:
y_p = ec_model.predict([x_val_batch, y_val_coarse_batch])

In [31]:
np_hard_normalized_hard_iou_coef(y_val_diff_batch, y_p)

0.7022625662229478

# Apply Error Correction Networks (X only)

In [33]:
ec_model = get_ec_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
concatenate_28 (Concatenate)    (None, 64, 64, 6)    0           input_14[0][0]                   
                                                                 input_13[0][0]                   
__________________________________________________________________________________________________
conv2d_64 (Conv2D)              (None, 64, 64, 8)    776         concatenate_28[0][0]             
__________

In [34]:
n_epochs = 3
batch_size = 8
skip = 12
N_CLEAN = 500
best_val = 0

val_generator = generator.data_generator('dataset_parser/data.h5', 32, 'val', ignore_pedestrians=True)

x_val, y_val = next(val_generator)
y_val_coarse = coarse_model.predict(x_val)
x_val_batch, y_val_coarse_batch, y_val_diff_batch = get_ec_batch(
    x_val, y_val, y_val_coarse, input_window_size=input_window_size,
    output_window_size=output_window_size, skip=skip, diff=False)

for e in range(n_epochs):
    fine_generator = generator.data_generator('dataset_parser/data.h5', batch_size, 'train', 
                                              ignore_pedestrians=ignore_pedestrians)
    for i in range(N_CLEAN // batch_size):
        x, y_fine = next(fine_generator)
        y_coarse = coarse_model.predict(x)
        x_batch, y_coarse_batch, y_diff_batch = get_ec_batch(x, y_fine, y_coarse, input_window_size=input_window_size,
                                                             output_window_size=output_window_size, skip=skip, diff=False,
                                                             balanced=True)
        y_coarse_batch = np.random.random(y_coarse_batch.shape)
#         metrics = ec_model.train_on_batch(y_coarse_batch, y_diff_batch)
        metrics = ec_model.train_on_batch([x_batch, y_coarse_batch], y_diff_batch)  # x and y
        print(i, metrics)
    
#     metrics = ec_model.evaluate(y_val_coarse_batch, y_val_diff_batch)  # y only
    metrics = ec_model.evaluate([x_val_batch, y_val_coarse_batch], y_val_diff_batch)  # x and y 
    print('\nEpoch {}, batch {}, Validation metrics (loss, dice_coef, iou, hard_iou): {}'.format(e, i, metrics))
    if metrics[-1] > best_val:
        best_val = metrics[-1]
        ec_model.save('best-ec-model-5.h5')
        print('Model saved')

500
500
2975
2975
0 [1.0800884, 0.34147757, 0.19731233, 0.16304842866558406]
1 [1.0539037, 0.379826, 0.20197394, 0.14859395697760341]
2 [1.0841362, 0.40560898, 0.21083176, 0.15870741801973892]
3 [1.0737064, 0.39227805, 0.19692558, 0.15362673186634065]
4 [1.0046452, 0.3918818, 0.21181743, 0.1719836400817996]
5 [0.9800161, 0.3879606, 0.2064234, 0.18343558282208589]
6 [0.99490035, 0.38097125, 0.2011745, 0.12927191679049033]
7 [1.0228016, 0.3956398, 0.21124557, 0.18164850884548414]
8 [1.0911398, 0.37739447, 0.21197522, 0.24264082364771783]
9 [1.1113992, 0.36623886, 0.20122373, 0.1588516883230614]
10 [0.9716969, 0.40536278, 0.21976185, 0.23264596216057892]
11 [0.9561418, 0.40202078, 0.2125301, 0.20847812527844606]
12 [0.99690527, 0.39217684, 0.21341531, 0.24759422711367787]
13 [1.0044606, 0.38741308, 0.20934795, 0.24375608210669744]
14 [0.9801368, 0.39617667, 0.21151033, 0.200502289182737]
15 [0.9134099, 0.41574067, 0.21084249, 0.17198847202280626]
16 [0.95741904, 0.4093514, 0.21465842, 0.1

In [35]:
y_p = ec_model.predict([x_val_batch, y_val_coarse_batch])

In [36]:
np_hard_normalized_hard_iou_coef(y_val_diff_batch, y_p)

0.31132497244743584

In [37]:
np_hard_normalized_hard_iou_coef(y_val_diff_batch, y_p)

0.31132497244743584